In [6]:
import polars as pl

In [ ]:
from __future__ import annotations
import numpy as np
import pandas as pd
from math import log, sqrt, exp
from typing import Literal
from mzpricer import OptionType, option_greeks, option_price, StockPrice, TimeDuration

# --- math helpers ---
from math import erf
def _norm_cdf(x: np.ndarray | float) -> np.ndarray | float:
    # standard normal CDF via erf
    return 0.5 * (1.0 + erf(np.asarray(x) / np.sqrt(2.0)))

# def bsm_price_delta(
#     S: float, K: float, T: float, r: float, q: float, sigma: float, opt_type: Literal["C","P"]
# ) -> tuple[float, float]:
#     """
#     Black–Scholes–Merton European price and delta with continuous dividend yield q.
#     Units:
#       S,K in currency; T in years; r,q,sigma annualized (decimals).
#     """
#     if T <= 0 or sigma <= 0 or S <= 0 or K <= 0:
#         # handle edge cases conservatively
#         if opt_type == "C":
#             intrinsic = max(0.0, S - K)
#             delta = 1.0 if S > K else (0.0 if S < K else 0.5)
#         else:
#             intrinsic = max(0.0, K - S)
#             delta = -1.0 if S < K else (0.0 if S > K else -0.5)
#         return intrinsic, delta

#     sig_sqrtT = sigma * sqrt(T)
#     d1 = (log(S / K) + (r - q + 0.5 * sigma * sigma) * T) / sig_sqrtT
#     d2 = d1 - sig_sqrtT

#     Nd1 = _norm_cdf(d1)
#     Nd2 = _norm_cdf(d2)
#     Nmd1 = _norm_cdf(-d1)
#     Nmd2 = _norm_cdf(-d2)

#     disc_r = exp(-r * T)
#     disc_q = exp(-q * T)

#     if opt_type == "C":
#         price = S * disc_q * Nd1 - K * disc_r * Nd2
#         delta = disc_q * Nd1
#     else:
#         price = K * disc_r * Nmd2 - S * disc_q * Nmd1
#         delta = disc_q * (Nd1 - 1.0)  # or -disc_q*N(-d1)

#     return price, delta

def _yearfrac_act365(t0: pd.Timestamp, t1: pd.Timestamp) -> float:
    return max((t1 - t0).total_seconds(), 0) / (365.0 * 24 * 3600)

def enrich_chain_with_bsm(df: pd.DataFrame) -> pd.DataFrame:
    """
    Expects columns:
      Spot, Type ('C'/'P'), Strike, Expiry, ValuationTime, Rate, DividendYield, Vol30d, Mid, ContractMultiplier
    Produces:
      Fair, Delta, Mispricing (= Mid - Fair), FairNotional (= Fair * ContractMultiplier), etc.
    """
    out = df.copy()

    # Parse times (robust to date-only strings)
    vt = pd.to_datetime(out["ValuationTime"], utc=False, errors="coerce")
    ex = pd.to_datetime(out["Expiry"], utc=False, errors="coerce")

    # Time to expiry (ACT/365)
    T = np.array([_yearfrac_act365(t0, t1) if (pd.notna(t0) and pd.notna(t1)) else np.nan
                  for t0, t1 in zip(vt, ex)])

    T = [TimeDuration(t, 365) for t in T]
    # Vectorize computation
    K = out["Strike"].astype(float).to_numpy()
    r = out["Rate"].astype(float).to_numpy()
    sig = out["Vol30d"].astype(float).to_numpy()
    typ = out["Type"].astype(str).str.upper().to_numpy()
    s_prime = [StockPrice(spot_price=spot, dividend_amount=0.0, time_to_dividend_days=0.0, rate=0.0).s_prime() for spot in out["Spot"].astype(float).to_numpy()]

    fair = np.empty(len(out))
    delta = np.empty(len(out))
    ttype = [OptionType.Call if t == 'C' else OptionType.Put for t in typ]
  
    # (s_prime, K, duration, R, SIGMA, OptionType.Call)
    p = option_price(s_prime, K, T, r, sig, ttype)
    greeks = option_greeks(s_prime, K, T, r, sig, ttype)
    
    out["Fair"] = p
    out["Delta"] = [d['delta'] for d in greeks]
    out["Mispricing"] = out["Mid"].astype(float) - out["Fair"]
    out["FairNotional"] = out["Fair"] * out.get("ContractMultiplier", 100)

    # Optional sanity check fields
    out["Forward"] = out["Spot"] * np.exp((out["Rate"] - out["DividendYield"]) * out["T_years"])

    return out

# --- Example usage ---
# df = pd.read_csv("options_chain_template.csv")
# enriched = enrich_chain_with_bsm(df)
# enriched.to_csv("options_chain_fair.csv", index=False)


In [21]:

def _fmt_date_series(col, fmt="%y-%m-%d"):
    s = pd.to_datetime(col, errors="coerce")
    # format elementwise; skip NaT
    return s.map(lambda x: x.strftime(fmt) if pd.notna(x) else None)

In [22]:

from itertools import product

def _to_datetime_safe(s: pd.Series) -> pd.Series:
    """Coerce Expiry-like series to pandas datetime."""
    s2 = s.copy()

    # If integers like 20230825, convert to string first
    mask_int = pd.api.types.is_integer_dtype(s2) | pd.api.types.is_float_dtype(s2)
    if mask_int:
        s2 = s2.astype("Int64").astype(str)

    # Try several common formats quickly; fallback to generic parser
    def _parse_one(x):
        if pd.isna(x):
            return pd.NaT
        x = str(x).strip()
        for fmt in ("%Y-%m-%d", "%Y%m%d", "%y-%m-%d", "%m/%d/%Y"):
            try:
                return pd.to_datetime(x, format=fmt)
            except Exception:
                pass
        # Last resort: flexible parser
        try:
            return pd.to_datetime(x, errors="coerce", utc=False)
        except Exception:
            return pd.NaT

    return s2.map(_parse_one)

def _format_expiry_cols(res: pd.DataFrame, cols=("Expiry", "LongExpiry", "ShortExpiry")) -> pd.DataFrame:
    res = res.copy()
    for c in cols:
        if c in res.columns:
            dt = _to_datetime_safe(res[c])
            res[c] = dt.dt.strftime("%y-%m-%d")
    return res


def _prep(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # Executable-price edges
    df["EdgeBuy"]  = (df["Fair"] - df["Ask"]) * df["ContractMultiplier"]     # long leg
    df["EdgeSell"] = (df["Bid"]  - df["Fair"]) * df["ContractMultiplier"]     # short leg
    
    # Premiums in $
    df["PayAsk"] = df["Ask"] * df["ContractMultiplier"]
    df["GetBid"] = df["Bid"] * df["ContractMultiplier"]
    
    # Useful flags
    df["IsBuyCandidate"]  = df["EdgeBuy"]  > 0
    df["IsSellCandidate"] = df["EdgeSell"] > 0

    return df

def _integer_ratio_for_delta(d_long, d_short, max_qty=50, tol=0.05):
    """
    Find small integer quantities (ql, qs) so ql*d_long + qs*d_short ~ 0.
    Returns best (ql, qs) by minimal |net delta|, then by smaller size.
    """
    best = None
    best_key = (1e9, 1e9)  # (abs_net_delta, total_size)
    for ql in range(1, max_qty+1):
        # qs chosen to offset delta approximately
        qs_float = abs(ql * d_long / d_short) if d_short != 0 else None
        if not qs_float or not np.isfinite(qs_float) or qs_float <= 0:
            continue
        for qs in {int(np.floor(qs_float)), int(np.ceil(qs_float))}:
            if qs < 1 or qs > max_qty:
                continue
            net_delta = ql*d_long + qs*d_short
            key = (abs(net_delta), ql+qs)
            if key < best_key:
                best_key = key
                best = (ql, qs, net_delta)
    if best and abs(best[2]) <= tol:
        return best[0], best[1], best[2]
    return None

def build_verticals(df: pd.DataFrame,
                    delta_tol=0.05,
                    max_qty=50,
                    min_liq_bid=0.01,
                    min_liq_ask=0.01):
    d = _prep(df)
    out = []
    for (tic, exp, opt_type), grp in d.groupby(["Ticker", "Expiry", "Type"]):
        # Liquidity filter (basic)
        g = grp[(grp["Bid"] >= min_liq_bid) & (grp["Ask"] >= min_liq_ask)].copy()
        if g.empty: 
            continue
        # Candidates
        longs  = g[g["IsBuyCandidate"]]   # buy underpriced
        shorts = g[g["IsSellCandidate"]]  # sell overpriced
        if longs.empty or shorts.empty:
            continue
        # Different strikes only
        longs = longs.rename(columns=lambda c: f"L_{c}")
        shorts = shorts.rename(columns=lambda c: f"S_{c}")
        for _, L in longs.iterrows():
            for _, S in shorts.iterrows():
                if L["L_Strike"] == S["S_Strike"]:
                    continue
                # Sizing by delta
                sizing = _integer_ratio_for_delta(L["L_Delta"], S["S_Delta"],
                                                  max_qty=max_qty, tol=delta_tol)
                if not sizing:
                    continue
                qL, qS, net_delta = sizing
                total_edge = qL*L["L_EdgeBuy"] + qS*S["S_EdgeSell"]
                net_premium = qL*L["L_PayAsk"] - qS*S["S_GetBid"]  # debit (>0) or credit (<0)
                # Compute cashflows explicitly
                long_cashflow  = - qL * L["L_Ask"] * L["L_ContractMultiplier"]
                short_cashflow = + qS * S["S_Bid"] * S["S_ContractMultiplier"]

                net_cashflow = long_cashflow + short_cashflow   # signed
                total_edge   = qL*L["L_EdgeBuy"] + qS*S["S_EdgeSell"]

                edge_eff = total_edge / (abs(net_cashflow) + 1e-6)  # per $ exchanged

                # Very rough margin proxy (customize for your broker/rules)
                # For credit verticals you could use width*multiplier*qty - credit; here keep simple:
                margin_proxy = abs(net_premium) + 1e-6  # avoid div-by-zero
                
                out.append({
                    "Strategy": "Vertical",
                    "Ticker": tic,
                    "Type": opt_type,
                    "Expiry": exp,
                    "LongStrike":  L["L_Strike"],
                    "ShortStrike": S["S_Strike"],
                    "qLong": qL,
                    "qShort": qS,
                    "NetDelta": net_delta,
                    "TotalEdge_$": total_edge,
                    "Edge_per_$Premium": total_edge / margin_proxy,
                    "NetPremium_$": net_premium,
                    "NetCashflow": net_cashflow,
                    "Long_leg_desc":  f"BUY {qL} @ Ask {L['L_Ask']:.2f} (Fair {L['L_Fair']:.2f}, Δ {L['L_Delta']:.3f})",
                    "Short_leg_desc": f"SELL {qS} @ Bid {S['S_Bid']:.2f} (Fair {S['S_Fair']:.2f}, Δ {S['S_Delta']:.3f})"
                })
    res = pd.DataFrame(out)
    if not res.empty:
        res = res.sort_values(["TotalEdge_$","Edge_per_$Premium"], ascending=False).reset_index(drop=True)
    return res

def build_calendars(df: pd.DataFrame,
                    delta_tol=0.05,
                    max_qty=50,
                    min_liq_bid=0.01,
                    min_liq_ask=0.01):
    d = _prep(df)
    out = []
    for (tic, strike, opt_type), grp in d.groupby(["Ticker", "Strike", "Type"]):
        g = grp[(grp["Bid"] >= min_liq_bid) & (grp["Ask"] >= min_liq_ask)].copy()
        if g["Expiry"].nunique() < 2:
            continue
        # Pick underpriced long in one expiry vs overpriced short in another expiry
        longs  = g[g["IsBuyCandidate"]].rename(columns=lambda c: f"L_{c}")
        shorts = g[g["IsSellCandidate"]].rename(columns=lambda c: f"S_{c}")
        if longs.empty or shorts.empty:
            continue
        for _, L in longs.iterrows():
            for _, S in shorts.iterrows():
                if L["L_Expiry"] == S["S_Expiry"]:
                    continue
                sizing = _integer_ratio_for_delta(L["L_Delta"], S["S_Delta"],
                                                  max_qty=max_qty, tol=delta_tol)
                if not sizing:
                    continue
                qL, qS, net_delta = sizing
                total_edge = qL*L["L_EdgeBuy"] + qS*S["S_EdgeSell"]
                net_premium = qL*L["L_PayAsk"] - qS*S["S_GetBid"]
                margin_proxy = abs(net_premium) + 1e-6

                # Format expiry nicely
                long_exp_str  = pd.to_datetime(L["L_Expiry"]).strftime("%Y-%m-%d")
                short_exp_str = pd.to_datetime(S["S_Expiry"]).strftime("%Y-%m-%d")

                out.append({
                    "Strategy": "Calendar",
                    "Ticker": tic,
                    "Type": opt_type,
                    "Strike": strike,
                    "LongExpiry":  long_exp_str,
                    "ShortExpiry": short_exp_str,
                    "qLong": qL,
                    "qShort": qS,
                    "NetDelta": net_delta,
                    "TotalEdge_$": total_edge,
                    "Edge_per_$Premium": total_edge / margin_proxy,
                    "NetPremium_$": net_premium,
                    "Long_leg_desc":  f"BUY {qL} {long_exp_str} @ {L['L_Ask']:.2f} "
                                    f"(Fair {L['L_Fair']:.2f}, Δ {L['L_Delta']:.3f})",
                    "Short_leg_desc": f"SELL {qS} {short_exp_str} @ {S['S_Bid']:.2f} "
                                    f"(Fair {S['S_Fair']:.2f}, Δ {S['S_Delta']:.3f})"
                })
    res = pd.DataFrame(out)
    if not res.empty:
        res = res.sort_values(["TotalEdge_$","Edge_per_$Premium"], ascending=False).reset_index(drop=True)
    return res

def build_risk_reversals(df: pd.DataFrame,
                         delta_tol=0.05,
                         max_qty=50,
                         min_liq_bid=0.01,
                         min_liq_ask=0.01):
    d = _prep(df)
    out = []
    for (tic, exp), grp in d.groupby(["Ticker", "Expiry"]):
        g = grp[(grp["Bid"] >= min_liq_bid) & (grp["Ask"] >= min_liq_ask)].copy()
        if g.empty: 
            continue
        calls = g[g["Type"]=="C"]
        puts  = g[g["Type"]=="P"]
        # Two classic constructions: sell rich put / buy cheap call, or the opposite
        for direction in ("SellPutBuyCall", "SellCallBuyPut"):
            if direction == "SellPutBuyCall":
                longs  = calls[calls["IsBuyCandidate"]].rename(columns=lambda c: f"L_{c}")
                shorts = puts[ puts["IsSellCandidate"]].rename(columns=lambda c: f"S_{c}")
            else:
                longs  = puts[ puts["IsBuyCandidate"]].rename(columns=lambda c: f"L_{c}")
                shorts = calls[calls["IsSellCandidate"]].rename(columns=lambda c: f"S_{c}")
            if longs.empty or shorts.empty:
                continue
            for _, L in longs.iterrows():
                for _, S in shorts.iterrows():
                    sizing = _integer_ratio_for_delta(L["L_Delta"], S["S_Delta"],
                                                      max_qty=max_qty, tol=delta_tol)
                    if not sizing:
                        continue
                    qL, qS, net_delta = sizing
                    total_edge = qL*L["L_EdgeBuy"] + qS*S["S_EdgeSell"]
                    net_premium = qL*L["L_PayAsk"] - qS*S["S_GetBid"]
                    margin_proxy = abs(net_premium) + 1e-6
                    out.append({
                        "Strategy": f"RiskReversal_{direction}",
                        "Ticker": tic,
                        "Expiry": exp,
                        "qLong": qL,
                        "qShort": qS,
                        "LongLeg":  f"{L['L_Type']}{L['L_Strike']}",
                        "ShortLeg": f"{S['S_Type']}{S['S_Strike']}",
                        "NetDelta": net_delta,
                        "TotalEdge_$": total_edge,
                        "Edge_per_$Premium": total_edge / margin_proxy,
                        "NetPremium_$": net_premium,
                        "Long_leg_desc":  f"BUY {qL} @ {L['L_Ask']:.2f} (Fair {L['L_Fair']:.2f}, Δ {L['L_Delta']:.3f})",
                        "Short_leg_desc": f"SELL {qS} @ {S['S_Bid']:.2f} (Fair {S['S_Fair']:.2f}, Δ {S['S_Delta']:.3f})"
                    })
    res = pd.DataFrame(out)
    if not res.empty:
        res = res.sort_values(["TotalEdge_$","Edge_per_$Premium"], ascending=False).reset_index(drop=True)
    return res

In [23]:
cols = [
    "Ticker",            # e.g., AAPL
    "ValuationTime",     # ISO8601, e.g., 2025-09-11T15:30:00
    "Spot",              # Underlying spot price S
    "Type",              # 'C' for call, 'P' for put
    "Strike",            # K
    "Expiry",            # ISO date or datetime for option expiration
    "Rate",              # risk-free cont. comp. rate r (annualized, decimal)
    "DividendYield",     # continuous dividend yield q (annualized, decimal)
    "Vol30d",            # annualized vol sigma (decimal, e.g., 0.25)
    "ContractMultiplier",# usually 100
    "Bid",               # market bid
    "Ask",               # market ask
    "Mid"                # market mid you will compare to fair
]

In [24]:
df = pl.read_csv("UnderlyingOptionsEODCalcs_2023-08-25_no_cgi_subscription.csv")
df = pl.read_csv("UnderlyingOptionsEODCalcs_2023-08-25_cgi_or_historical.csv")

In [25]:
import pandas as pd 
def get_price_series(df, ticker_symbol, prefer_adj=True):
    if isinstance(df.columns, pd.MultiIndex):
        cols = df.columns
        if prefer_adj and ('Adj Close', ticker_symbol) in cols:
            return df[('Adj Close', ticker_symbol)].rename('Adj Close')
        if ('Close', ticker_symbol) in cols:
            return df[('Close', ticker_symbol)].rename('Close')
    else:
        if prefer_adj and 'Adj Close' in df.columns:
            return df['Adj Close']
        if 'Close' in df.columns:
            return df['Close']
    raise KeyError("Could not find a price column ('Adj Close' or 'Close').")



In [26]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define the ticker symbol and date range
ticker_symbol = "TSLA"
start_date = "2023-06-25"
end_date = "2023-08-25"

# Download the data
tsla_data = yf.download(ticker_symbol, start=start_date, end=end_date)
price = get_price_series(tsla_data, 'TSLA', prefer_adj=True)
# Ensure returns are based on adjusted close
tsla_data['Return'] = price.pct_change()

# Compute 30-day rolling volatility (annualized)
# As of 2023-08-23 close
window = 30
trading_days = 252  # typical number of trading days in a year
tsla_data['Volatility_30d'] = (
    tsla_data['Return'].rolling(window).std() * np.sqrt(trading_days)
)

# Get the volatility on 2023-08-23
vol_20230823 = tsla_data.loc["2023-08-23", "Volatility_30d"]
vol_20230823

/var/folders/8_/y69hhps94q5cvrnr_2hlmnq80000gp/T/ipykernel_89214/1607415334.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  tsla_data = yf.download(ticker_symbol, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


Ticker
    0.487469
Name: 2023-08-23 00:00:00, dtype: float64

In [29]:
"""
Assumptions:
1545 marks for underlying
1545 marks for options
spot is mid of underlying at close
borrow rate is made up
temp_div is 0 for TSLA so ignore for now
"""
input_data = (df.filter(pl.col("underlying_symbol") == "TSLA")
 .with_columns(
    pl.datetime(
        year=2023,
        month=8,
        day=23,
        hour=16,
        minute=0,
        second=0,
        time_zone="America/New_York",
 ).alias("ValuationTime"),
 ((pl.col("underlying_ask_1545") + pl.col("underlying_bid_1545"))/2).alias("underlying_mid_1545"),
 (
     pl.col("expiration").str.to_datetime(format="%Y-%m-%d", time_zone="America/New_York") + pl.duration(hours=14)
 ).dt.to_string("iso"),
 pl.lit(100).alias('ContractMultiplier'),
 pl.lit(.05).alias("temp_borrow_rate"),
 pl.lit(0.0).alias("temp_div"),
 pl.lit(vol_20230823[0]).alias("temp_30dvol"),
 ((pl.col('ask_1545') + pl.col('bid_1545'))/2).alias('mid_1545')
 )

 .rename({
     "underlying_symbol": "Ticker", 
    "underlying_mid_1545": "Spot",             
    "option_type": "Type",             
    "strike": "Strike",           
    "expiration": "Expiry",           
    "temp_borrow_rate": "Rate",             
    "temp_div": "DividendYield",    
    "temp_30dvol": "Vol30d",           
    "ContractMultiplier": "ContractMultiplier",
    "bid_1545": "Bid",              
    "ask_1545": "Ask",              
    "mid_1545": "Mid"               
 })
 .select(cols)
)

/var/folders/8_/y69hhps94q5cvrnr_2hlmnq80000gp/T/ipykernel_89214/4020078671.py:27: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pl.lit(vol_20230823[0]).alias("temp_30dvol"),


In [30]:
output_df = enrich_chain_with_bsm(input_data.to_pandas())

/var/folders/8_/y69hhps94q5cvrnr_2hlmnq80000gp/T/ipykernel_89214/651943145.py:67: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  ex = pd.to_datetime(out["Expiry"], utc=False, errors="coerce")


TypeError: argument 'precision': 'list' object cannot be interpreted as an integer

In [16]:
verts = build_verticals(output_df)
riskys = build_risk_reversals(output_df)

In [17]:
verts

,Strategy,Ticker,Type,Expiry,LongStrike,ShortStrike,qLong,qShort,NetDelta,TotalEdge_$,Edge_per_$Premium,NetPremium_$,NetCashflow,Long_leg_desc,Short_leg_desc
0,Vertical,TSLA,P,2023-09-01 14:00:00.000000-04:00,205.0,187.5,1,30,-0.041545,111.346990,1.040626e+00,-107.0,107.0,"BUY 1 @ Ask 0.13 (Fair 0.15, Δ -0.021)","SELL 30 @ Bid 0.04 (Fair 0.00, Δ -0.001)"
1,Vertical,TSLA,P,2023-09-01 14:00:00.000000-04:00,205.0,190.0,1,17,-0.041988,75.666494,1.050924e+00,-72.0,72.0,"BUY 1 @ Ask 0.13 (Fair 0.15, Δ -0.021)","SELL 17 @ Bid 0.05 (Fair 0.01, Δ -0.001)"
2,Vertical,TSLA,C,2023-09-01 14:00:00.000000-04:00,282.5,310.0,1,43,0.029392,36.502342,1.073598e+00,-34.0,34.0,"BUY 1 @ Ask 0.09 (Fair 0.09, Δ 0.015)","SELL 43 @ Bid 0.01 (Fair 0.00, Δ 0.000)"
3,Vertical,TSLA,P,2023-09-01 14:00:00.000000-04:00,205.0,192.5,1,9,-0.040348,36.069241,1.127164e+00,-32.0,32.0,"BUY 1 @ Ask 0.13 (Fair 0.15, Δ -0.021)","SELL 9 @ Bid 0.05 (Fair 0.01, Δ -0.002)"
4,Vertical,TSLA,P,2023-09-01 14:00:00.000000-04:00,205.0,195.0,1,5,-0.039094,21.301376,1.253022e+00,-17.0,17.0,"BUY 1 @ Ask 0.13 (Fair 0.15, Δ -0.021)","SELL 5 @ Bid 0.06 (Fair 0.02, Δ -0.004)"
5,Vertical,TSLA,C,2023-09-01 14:00:00.000000-04:00,280.0,305.0,1,27,0.039326,20.277102,1.192771e+00,-17.0,17.0,"BUY 1 @ Ask 0.10 (Fair 0.13, Δ 0.020)","SELL 27 @ Bid 0.01 (Fair 0.00, Δ 0.001)"
6,Vertical,TSLA,C,2023-09-01 14:00:00.000000-04:00,280.0,300.0,1,13,0.039257,18.802836,1.175177e+00,-16.0,16.0,"BUY 1 @ Ask 0.10 (Fair 0.13, Δ 0.020)","SELL 13 @ Bid 0.02 (Fair 0.01, Δ 0.002)"
7,Vertical,TSLA,C,2023-09-01 14:00:00.000000-04:00,282.5,305.0,1,20,0.029290,13.210639,1.200967e+00,-11.0,11.0,"BUY 1 @ Ask 0.09 (Fair 0.09, Δ 0.015)","SELL 20 @ Bid 0.01 (Fair 0.00, Δ 0.001)"
8,Vertical,TSLA,P,2023-09-01 14:00:00.000000-04:00,205.0,197.5,1,3,-0.038686,12.062345,1.507793e+00,-8.0,8.0,"BUY 1 @ Ask 0.13 (Fair 0.15, Δ -0.021)","SELL 3 @ Bid 0.07 (Fair 0.03, Δ -0.006)"
9,Vertical,TSLA,C,2023-09-01 14:00:00.000000-04:00,282.5,300.0,1,9,0.028289,11.341425,1.260158e+00,-9.0,9.0,"BUY 1 @ Ask 0.09 (Fair 0.09, Δ 0.015)","SELL 9 @ Bid 0.02 (Fair 0.01, Δ 0.002)"


In [18]:
calendars = build_calendars(output_df)


In [19]:
calendars

""


In [20]:
riskys

,Strategy,Ticker,Expiry,qLong,qShort,LongLeg,ShortLeg,NetDelta,TotalEdge_$,Edge_per_$Premium,NetPremium_$,Long_leg_desc,Short_leg_desc
0,RiskReversal_SellPutBuyCall,TSLA,2024-01-19 14:00:00.000000-05:00,44,21,C260.0,P741.67,-0.015008,27740.872333,0.029134,-952170.0,"BUY 44 @ 22.95 (Fair 22.99, Δ 0.477)","SELL 21 @ 501.50 (Fair 488.36, Δ -1.000)"
1,RiskReversal_SellPutBuyCall,TSLA,2024-01-19 14:00:00.000000-05:00,44,21,C260.0,P733.33,-0.014087,27364.388976,0.029278,-934635.0,"BUY 44 @ 22.95 (Fair 22.99, Δ 0.477)","SELL 21 @ 493.15 (Fair 480.19, Δ -1.000)"
2,RiskReversal_SellPutBuyCall,TSLA,2024-01-19 14:00:00.000000-05:00,44,21,C260.0,P716.67,-0.011830,26674.432381,0.029649,-899670.0,"BUY 44 @ 22.95 (Fair 22.99, Δ 0.477)","SELL 21 @ 476.50 (Fair 463.87, Δ -1.000)"
3,RiskReversal_SellPutBuyCall,TSLA,2024-01-19 14:00:00.000000-05:00,44,21,C260.0,P708.33,-0.010448,26297.014572,0.029811,-882135.0,"BUY 44 @ 22.95 (Fair 22.99, Δ 0.477)","SELL 21 @ 468.15 (Fair 455.70, Δ -0.999)"
4,RiskReversal_SellPutBuyCall,TSLA,2024-01-19 14:00:00.000000-05:00,44,21,C260.0,P725.0,-0.013035,26232.065045,0.028626,-916365.0,"BUY 44 @ 22.95 (Fair 22.99, Δ 0.477)","SELL 21 @ 484.45 (Fair 472.03, Δ -1.000)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18468,RiskReversal_SellPutBuyCall,TSLA,2023-08-25 14:00:00.000000-04:00,1,1,C85.0,P272.5,0.000082,0.368889,0.000031,11925.0,"BUY 1 @ 153.35 (Fair 153.35, Δ 1.000)","SELL 1 @ 34.10 (Fair 34.10, Δ -1.000)"
18469,RiskReversal_SellPutBuyCall,TSLA,2023-08-25 14:00:00.000000-04:00,1,1,C80.0,P275.0,0.000028,0.314243,0.000026,12175.0,"BUY 1 @ 158.35 (Fair 158.35, Δ 1.000)","SELL 1 @ 36.60 (Fair 36.60, Δ -1.000)"
18470,RiskReversal_SellPutBuyCall,TSLA,2023-08-25 14:00:00.000000-04:00,1,1,C85.0,P270.0,0.000227,0.272055,0.000022,12175.0,"BUY 1 @ 153.35 (Fair 153.35, Δ 1.000)","SELL 1 @ 31.60 (Fair 31.60, Δ -1.000)"
18471,RiskReversal_SellPutBuyCall,TSLA,2023-08-25 14:00:00.000000-04:00,1,1,C80.0,P272.5,0.000082,0.237628,0.000019,12425.0,"BUY 1 @ 158.35 (Fair 158.35, Δ 1.000)","SELL 1 @ 34.10 (Fair 34.10, Δ -1.000)"


In [21]:
output_df.to_clipboard()

In [22]:
verts.to_clipboard()

In [54]:
riskys.to_csv('tsla_risk_reveral.csv')

In [23]:
riskys.to_clipboard()